In [30]:
import numpy as np 
import pandas as pd 

# Seteo del mes snapshot

In [31]:
mes_snapshot = 9  #9 para train 12 para scoreo

# Lectura de los csv

In [32]:
df_pageviews = pd.read_csv("./data/pageviews.csv",
                   parse_dates=["FEC_EVENT"])
df_devicedata = pd.read_csv("./data/device_data.csv",
                   parse_dates=["FEC_EVENT"])
df_conversiones = pd.read_csv("./data/conversiones.csv")

In [33]:
#Se eliminan los registros que no deben ser considerados.
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month <= mes_snapshot]
df_pageviews = df_pageviews[df_pageviews['FEC_EVENT'].dt.month > mes_snapshot - 9]

df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month <= mes_snapshot]
df_devicedata = df_devicedata[df_devicedata['FEC_EVENT'].dt.month > mes_snapshot - 9]

# Creación del universo

In [18]:
def universo_train(df):
    universo = pd.DataFrame()
    universo['user_id'] = df.USER_ID.unique()
    universo['snapshot_mes'] = mes_snapshot
    return universo

In [27]:
universo = universo_train(df_pageviews)

# Features

In [ ]:
def feature_X(df):
    

In [ ]:
def feature_X1(df):

# Armar dataset

In [ ]:
universo = feature_X(universo)
universo = feature_X1(universo)

# Target

In [28]:
def target(df):
    df = pd.merge(df, df_conversiones, how='left', left_on=['user_id'], right_on=['USER_ID'])
    df.mes.fillna(value= 99, inplace=True)
    df = df.astype({'mes': 'int32'}) 
    
    df['target'] = 0
    df.loc[df.mes == df.snapshot_mes +1, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +2, 'target'] = 1
    df.loc[df.mes == df.snapshot_mes +3, 'target'] = 1

    return (df.groupby(['user_id', 'snapshot_mes'])['target'].max()).to_frame().reset_index()

In [29]:
target(universo)

,user_id,snapshot_mes,target
0,0,9,0
1,1,9,0
2,2,9,0
3,3,9,0
4,4,9,0
5,5,9,0
6,6,9,0
7,7,9,0
8,8,9,0
9,9,9,0
